# Lectura de ficheros easyMoney

## Import Libraries

In [ ]:
import numpy as np 
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
from sklearn import preprocessing

## Import Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Products

In [ ]:
ca_file = '/content/drive/MyDrive/Captstone/data/commercial_activity_df.csv'

In [ ]:
products_file = '/content/drive/MyDrive/Captstone/data/products_df.csv'

In [ ]:
sd_file = '/content/drive/MyDrive/Captstone/data/sociodemographic_df.csv'

In [ ]:
commercial = pd.read_csv(ca_file)

In [ ]:
products = pd.read_csv(products_file)

In [ ]:
sociodemographic = pd.read_csv(sd_file)

In [ ]:
products.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   long_term_deposit   int64  
 9   em_account_pp       int64  
 10  credit_card         int64  
 11  payroll             float64
 12  pension_plan        float64
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
dtypes: float64(2), int64(15), object(1)
memory usage: 818.9+ MB


In [ ]:
#redefinimos el index con la columna Unnamed

commercial.set_index('Unnamed: 0', inplace = True)

In [ ]:
#redefinimos el index con la columna Unnamed

products.set_index('Unnamed: 0', inplace = True)

In [ ]:
#redefinimos el index con la columna Unnamed

sociodemographic.set_index('Unnamed: 0', inplace = True)

In [ ]:
#products.head(3)

In [ ]:
# clientes unicos
products['pk_cid'].nunique()

456373

In [ ]:
#Vemos que las fechas van desde el 1 de enero del 2018 hasta el 28 de mayo del 2019. Es decir 18 meses y no hay valores nulos
#podemos analizar la permanencia de los clientes
#los clientes nuevos por mes y año
#los clientes que se fueron 
#el producto mas adquirido por mes y año
products['pk_partition'].value_counts(dropna=False)

2019-05-28    442995
2019-04-28    439627
2019-03-28    436183
2019-02-28    431727
2019-01-28    426875
2018-12-28    422481
2018-11-28    416387
2018-10-28    402300
2018-09-28    375323
2018-08-28    352922
2018-07-28    339339
2018-06-28    252104
2018-05-28    249926
2018-04-28    247463
2018-03-28    245258
2018-02-28    242521
2018-01-28    239493
Name: pk_partition, dtype: int64

In [ ]:
#products[products['pk_cid'] == 1375586]

In [ ]:
#products[products['pk_partition'] == '2019-05-28'].describe().T

### Generamos valores de diferencias

In [ ]:
products_sorted = products.sort_values(by=['pk_cid', 'pk_partition'])

In [ ]:
#products_sorted.head(20)

In [ ]:
#Que estado tenia en el mes anterior
products_sorted['debit_card_previous_month'] = products_sorted.groupby('pk_cid')['debit_card'].shift(1)

In [ ]:
lista_productos = ['short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount'] 

In [ ]:
products.shape

(5962924, 17)

In [ ]:
for productos in lista_productos:
 products_sorted[productos + '_previous_month' ] = products_sorted.groupby('pk_cid')[productos].shift(1)
 products_sorted['dif_' + productos] = products_sorted[productos] - products_sorted[productos + '_previous_month']

In [ ]:
#products_sorted.head()

In [ ]:
#Venta o cancelacion
products_sorted['dif_debit_card'] = products_sorted['debit_card'] - products_sorted['debit_card_previous_month']

In [ ]:
products_sorted.shape

(5962924, 45)

In [ ]:
products_sorted.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount',
       'debit_card_previous_month', 'short_term_deposit_previous_month',
       'dif_short_term_deposit', 'loans_previous_month', 'dif_loans',
       'mortgage_previous_month', 'dif_mortgage', 'funds_previous_month',
       'dif_funds', 'securities_previous_month', 'dif_securities',
       'long_term_deposit_previous_month', 'dif_long_term_deposit',
       'em_account_pp_previous_month', 'dif_em_account_pp',
       'credit_card_previous_month', 'dif_credit_card',
       'pension_plan_previous_month', 'dif_pension_plan',
       'payroll_account_previous_month', 'dif_payroll_account',
       'emc_account_previous_month', 'dif_emc_account', 'dif_debit_card',
       'em_account_p_previous_month', 'dif_em_account_p',
 

In [ ]:
products_sorted['dif_debit_card'].value_counts()

0.00     5377974
1.00       73053
-1.00      55524
Name: dif_debit_card, dtype: int64

In [ ]:
#Por mes cuantos clientes dejaron de tener el producto (-1)
def clientes_baja_productos(df, column):
 return df[df['dif_'+ column] == -1].pivot_table(index='pk_partition', values= 'dif_' + column, aggfunc=[len])
  

In [ ]:
#Aca probamos con tarjeta de debito, despues continuamos con los otros productos y con aquellos que se dieron de alta. 
#clientes_baja_productos(products_sorted, 'debit_card')

In [ ]:
 products_sorted[products_sorted['dif_debit_card'] == -1].pivot_table(index='pk_partition', values='dif_debit_card', aggfunc=[len])

,len
,dif_debit_card
pk_partition,
2018-02-28,2913
2018-03-28,2432
2018-04-28,2742
2018-05-28,3013
2018-06-28,2754
2018-07-28,3373
2018-08-28,4010
2018-09-28,2792


In [ ]:
for producto in lista_productos:
  products[producto] = products[producto].astype('category')

In [ ]:
products.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 13647308
Data columns (total 17 columns):
 #   Column              Dtype   
---  ------              -----   
 0   pk_cid              int64   
 1   pk_partition        object  
 2   short_term_deposit  category
 3   loans               category
 4   mortgage            category
 5   funds               category
 6   securities          category
 7   long_term_deposit   category
 8   em_account_pp       category
 9   credit_card         category
 10  payroll             float64 
 11  pension_plan        category
 12  payroll_account     category
 13  emc_account         category
 14  debit_card          category
 15  em_account_p        category
 16  em_acount           category
dtypes: category(14), float64(1), int64(1), object(1)
memory usage: 261.6+ MB


### se agrega una columna de renevue por cada producto, y la columna de total revenue

- 10€ por cada cuenta vendida
- 40€ por los productos de ahorro e inversión (planes, fondos, etc.) 
- 60€ por los de financiación (prestamos y tarjetas).
- Tarjeta de credito es un préstamo. Me da 60 euros
- Tarjeta debito es un servicio (considerar 10 euros)


In [ ]:
lista_servicios = ['em_account_pp',
        'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount']

lista_ahorro_inv = ['short_term_deposit',
       'funds', 'securities', 'long_term_deposit', 'pension_plan',]

lista_fincanciacion = [ 'loans', 'mortgage','credit_card', ]

In [ ]:
products_sorted['total_revenue'] = 0
for productos in lista_productos:
  if productos in lista_servicios:
    revenue = 10
  elif productos in lista_ahorro_inv:
    revenue = 40
  elif productos in lista_fincanciacion:
    revenue = 60
  products_sorted['revenue_' + productos] = 0
  products_sorted.loc[products_sorted[productos] == 1 ,'revenue_' + productos]=  revenue
  products_sorted['total_revenue'][products_sorted[productos] == 1] = products_sorted['total_revenue'] [products_sorted[productos] == 1] + revenue
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [ ]:
#products_sorted

In [ ]:
#products_sorted[products_sorted['pk_cid'] == 17457]

In [ ]:
df_cliente = products_sorted[products_sorted['pk_cid'] == 17457]

In [ ]:
df_cliente.to_csv(r'/content/drive/MyDrive/Captstone/data/df_cliente_3.csv',index = False, sep=';')

In [ ]:
# products_sorted.to_csv(r'/content/drive/MyDrive/Nuclio/Capstone/products_sorted.csv',index = False, sep=';')

In [ ]:
df = products_sorted[products_sorted['total_revenue']>0]

In [ ]:
df_filtrado = pd.DataFrame(products_sorted[products_sorted['total_revenue']>0])

In [ ]:
#df_filtrado

In [ ]:
df_filtrado.shape

(4841417, 60)

In [ ]:
lista_categorical = ['short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount',
       'debit_card_previous_month', 'short_term_deposit_previous_month',
       'loans_previous_month', 
       'mortgage_previous_month', 'funds_previous_month',
      'securities_previous_month',
       'long_term_deposit_previous_month', 
       'em_account_pp_previous_month','credit_card_previous_month', 
       'pension_plan_previous_month', 
       'payroll_account_previous_month', 
       'emc_account_previous_month', 
       'em_account_p_previous_month', 
       'em_acount_previous_month', 
       'revenue_short_term_deposit', 'revenue_loans', 'revenue_mortgage',
       'revenue_funds', 'revenue_securities', 'revenue_long_term_deposit',
       'revenue_em_account_pp', 'revenue_credit_card', 'revenue_pension_plan',
       'revenue_payroll_account', 'revenue_emc_account', 'revenue_debit_card',
       'revenue_em_account_p', 'revenue_em_acount']

In [ ]:
for producto in lista_categorical:
  df_filtrado[producto] = df_filtrado[producto].astype('category')

In [ ]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4841417 entries, 4078994 to 13336847
Data columns (total 60 columns):
 #   Column                             Dtype   
---  ------                             -----   
 0   pk_cid                             int64   
 1   pk_partition                       object  
 2   short_term_deposit                 category
 3   loans                              category
 4   mortgage                           category
 5   funds                              category
 6   securities                         category
 7   long_term_deposit                  category
 8   em_account_pp                      category
 9   credit_card                        category
 10  payroll                            category
 11  pension_plan                       category
 12  payroll_account                    category
 13  emc_account                        category
 14  debit_card                         category
 15  em_account_p                       categor

In [ ]:
df_filtrado.head(20)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,revenue_securities,revenue_long_term_deposit,revenue_em_account_pp,revenue_credit_card,revenue_pension_plan,revenue_payroll_account,revenue_emc_account,revenue_debit_card,revenue_em_account_p,revenue_em_acount
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
4078994,15891,2018-07-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
9401344,16203,2019-01-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
10425906,16203,2019-02-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
11170676,16203,2019-03-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
12221208,16203,2019-04-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
13026405,16203,2019-05-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
6909890,16502,2018-10-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
7450803,16502,2018-11-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
8618141,16502,2018-12-28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10


In [ ]:
new_df_filtrado = pd.merge(df_filtrado,commercial,how='left',on=["pk_cid", "pk_partition"])

In [ ]:
new_df_filtrado.shape

In [ ]:
final_df = pd.merge(new_df_filtrado,sociodemographic,how='left',on=["pk_cid", "pk_partition"])

In [ ]:
final_df.shape

In [ ]:
final_df.to_csv(r'C:\Users\FlorenciaPRANDI\Desktop\final_df.csv',index = False, sep=';')

In [ ]:
final_df[final_df['pk_cid']==808433]